In [1]:
import reg_mri
import os
from glob import glob
from utils import compute_mean_dice
import nibabel as nib
from scipy.spatial.distance import dice
import numpy as np
import itk
import SimpleITK as sitk
import scipy.ndimage
import scipy
import matplotlib.pyplot as plt
from transforms_dict import getRegistrationEvalInverseTransformForMRI, SaveTransformForMRI
from tqdm import tqdm
import monai
import subprocess

In [2]:
# Save Template with same Atlas header
# template_name = "dataset2/Atlas/Deformable_Feminad_template0.nii.gz"
# mask_name = "dataset2/Atlas/Deformable_Feminad_Mask_template0.nii.gz"
# atlas_name = "dataset2/Atlas/Norm_Atlas_MRI_128.nii.gz"
# 
# affine = nib.load(atlas_name).affine
# header = nib.load(atlas_name).header
# 
# template = nib.load(template_name).get_fdata()
# new_template = nib.Nifti1Image(template, affine, header)
# 
# outname_template = "dataset2/Atlas/Deformable_Feminad_template0_sameaffine.nii.gz"
# nib.save(new_template, outname_template)
# 
# mask = nib.load(mask_name).get_fdata()
# new_mask = nib.Nifti1Image(mask, affine, header)
# 
# outname_mask = "dataset2/Atlas/Deformable_Feminad_Mask_template0_sameaffine.nii.gz"
# nib.save(new_mask, outname_mask)

In [3]:
model = "registration/deformable/template_deformable_feminad_newloss_fixedaffine_lr1e-4_bending_trilinear_2-0-1.pth"
mris = sorted(glob(os.path.join('dataset2', 'Feminad', 'MRI_N4_Resample_Norm', "*.nii.gz")))
atlas_name = "dataset2/Atlas/Deformable_Feminad_template0_sameaffine.nii.gz"
outfolder = "DL_Trilinear"
sample_name = "output/Feminad/ANTS/N4_2_6516_Warp.nii.gz"
for i, mri in enumerate(mris):
    outname = "output/Feminad/" + outfolder + "/MRI_N4_Registration_Deformable/" + mri.split('/')[-1].split('.')[0]
    pred_image, ddfs = reg_mri.main(model, mri, outname, False, True, "deformable")
    
    moving_image = ddfs[0].cpu()
    
    affine_ddf = ddfs[1].cpu()    
    affine_ddf = affine_ddf.cpu().numpy()
    affine_ddf = np.transpose(affine_ddf, (2, 3, 4, 0, 1))
    #affine_ddf = scipy.ndimage.gaussian_filter(affine_ddf, sigma=1.0) 
    
    deformable_ddf = ddfs[2].cpu()
    deformable_ddf = deformable_ddf.cpu().numpy()
    deformable_ddf = np.transpose(deformable_ddf, (2, 3, 4, 0, 1))
    #deformable_ddf = scipy.ndimage.gaussian_filter(deformable_ddf, sigma=1.0) 
           
    affine = nib.load(sample_name).affine
    header = nib.load(sample_name).header 
    
    affine_image = nib.Nifti1Image(affine_ddf, affine, header)
    print(affine_ddf.shape)
    #outname_affine = "output/Feminad/" + outfolder + "/AffineWarp/" + mri.split('/')[-1].split('.')[0] + ".nii.gz"
    #nib.save(affine_image, outname_affine)
    #
    #deformable_image = nib.Nifti1Image(deformable_ddf, affine, header)
    #outname_deformable = "output/Feminad/" + outfolder + "/DeformableWarp/" + mri.split('/')[-1].split('.')[0] + ".nii.gz"
    #nib.save(deformable_image, outname_deformable)
    print(i, end='\r')
    break

    

=> Saved to output/Feminad/DL_Trilinear/MRI_N4_Registration_Deformable/Norm_Resampled_N4_12_6517_MRI.nii.gz
=> Saved to output/Feminad/DL_Trilinear/MRI_N4_Registration_Deformable/Norm_Resampled_N4_12_6517_Resample.nii.gz
=> Saved to output/Feminad/DL_Trilinear/MRI_N4_Registration_Deformable/Norm_Resampled_N4_12_6517_Resample_Reg.nii.gz
=> Saved to output/Feminad/DL_Trilinear/MRI_N4_Registration_Deformable/Norm_Resampled_N4_12_6517_Reg.nii.gz
1.5013134239999992
(128, 128, 128, 1, 3)


In [4]:
def getLog10Jacobian_deformable_ants(warp_img, outfolder):
    outdir = os.path.join('output', 'Feminad', outfolder, 'Log10Jacobian_Deformable')
    outname = outdir + '/Log10JacobianDeformable_' + warp_img.split('/')[-1].split('Warp')[0] + '.nii.gz'
    jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warp_img) + " " + str(outname) + " 1 1"
    subprocess.call(jacobian_command.split(" "))

In [5]:
def getLog10Jacobian_affine_ants(warp_img, outfolder):
    outdir = os.path.join('output', 'Feminad', outfolder, 'Log10Jacobian_Affine')
    outname = outdir + '/Log10JacobianAffine_' + warp_img.split('/')[-1].split('Warp')[0] + '.nii.gz'
    jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warp_img) + " " + str(outname) + " 1 1"
    subprocess.call(jacobian_command.split(" "))

In [6]:
def getLog10Jacobian_affinedeformable_ants(aff_log10jac, def_log10jac, outfolder):
    outdir = os.path.join('output', 'Feminad', outfolder, 'Log10Jacobian_AffineDeformable')
    outname = outdir + '/Log10JacobianAffineDeformable' + aff_log10jac.split('/')[-1].split('Log10JacobianAffine')[-1]
    combine_command = "ImageMath 3 " + str(outname) + " + " + str(aff_log10jac) + " " + str(def_log10jac)
    subprocess.call(combine_command.split(" "))

In [7]:
mridir = 'output/Feminad/' + outfolder + '/'
affine_warps = sorted(glob(os.path.join(mridir, 'AffineWarp', "*.nii.gz")))
deformable_warps = sorted(glob(os.path.join(mridir, 'DeformableWarp', "*.nii.gz")))
for i in range(len(affine_warps)):
    getLog10Jacobian_affine_ants(affine_warps[i], outfolder)    
    getLog10Jacobian_deformable_ants(deformable_warps[i], outfolder)  
    print(i+1, end='\r')

In [8]:
aff_log10jacs = sorted(glob(os.path.join(mridir, 'Log10Jacobian_Affine', "*.nii.gz")))
def_log10jacs = sorted(glob(os.path.join(mridir, 'Log10Jacobian_Deformable', "*.nii.gz")))
for i in range(len(aff_log10jacs)):
    getLog10Jacobian_affinedeformable_ants(aff_log10jacs[i], def_log10jacs[i], outfolder)
    print(i+1, end='\r')